In [1]:
import os
import sys
import importlib

# -----------------------------
# Paths (edit if needed)
# -----------------------------
repo_root = "/scratch/gilbreth/abelde/Thesis/StructureAwareGen/scripts/rcg"
config_path = "config/rdm/mocov3vitb_simplemlp_l12_w1536.yaml"  # relative to repo_root is fine
output_dir = os.environ.get("OUTPUT_DIR", "/scratch/gilbreth/abelde/rdm_debug_out")
imagenet_dir = os.environ.get("IMAGENET_DIR", "/scratch/gilbreth/abelde/datasets/imagenet")

# -----------------------------
# Put repo on import path
# -----------------------------
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

# If you already imported these in the notebook, reload to pick up edits/breakpoints
import main_rdm
importlib.reload(main_rdm)

# # -----------------------------
# # Debug tip:
# # For notebook debugging, DON'T use torch.distributed.launch.
# # Run single GPU / single process. That means:
# #   - remove/ignore --dist_url / --nnodes / --node_rank / --nproc_per_node
# # -----------------------------
# sys.argv = [
#     "main_rdm.py",
#     "--config", config_path,
#     "--batch_size", "128",
#     "--input_size", "256",
#     "--epochs", "200",
#     "--blr", "1e-6",
#     "--weight_decay", "0.01",
#     "--output_dir", output_dir,
#     "--data_path", imagenet_dir,

#     # Optional: make sure it doesn't try multi-process
#     # (one of these usually exists; keep only the ones your argparse supports)
#     "--world_size", "1",
#     "--local_rank", "0",
# ]

# # -----------------------------
# # Run
# # -----------------------------
# # Many repos have main() defined; if not, they run under `if __name__ == "__main__":`
# # We'll try main() first.
# if hasattr(main_rdm, "main"):
#     main_rdm.main()
# else:
#     # fallback: execute the file as a script
#     import runpy
#     runpy.run_path(os.path.join(repo_root, "main_rdm.py"), run_name="__main__")


2026-01-17 17:43:57.403571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768689837.530048 2992297 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768689837.564813 2992297 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768689837.882650 2992297 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768689837.882665 2992297 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768689837.882667 2992297 computation_placer.cc:177] computation placer alr

AssertionError: 

In [2]:
# Notebook cell: print versions of packages in your current conda env
import sys, subprocess, importlib
from importlib.metadata import version as dist_version, PackageNotFoundError

print("Python:", sys.version.split()[0])
print("Executable:", sys.executable)
print()

# Map pip/distribution names -> import names (when different)
PKGS = [
    ("python", None),
    ("pip", None),
    ("cudatoolkit", None),          # conda package; we'll show via `conda list` if available
    ("pytorch", "torch"),
    ("torchvision", "torchvision"),
    ("numpy", "numpy"),
    ("albumentations", "albumentations"),
    ("opencv-python", "cv2"),
    ("pudb", "pudb"),
    ("imageio", "imageio"),
    ("imageio-ffmpeg", None),       # dist name only
    ("pytorch-lightning", "pytorch_lightning"),
    ("torchmetrics", "torchmetrics"),
    ("setuptools", "setuptools"),
    ("omegaconf", "omegaconf"),
    ("test-tube", "test_tube"),
    ("streamlit", "streamlit"),
    ("einops", "einops"),
    ("more-itertools", "more_itertools"),
    ("timm", "timm"),
    ("protobuf", "google.protobuf"),
    ("submitit", "submitit"),
    ("diffusers", "diffusers"),
    ("accelerate", "accelerate"),
    ("huggingface-hub", "huggingface_hub"),
    ("transformers", "transformers"),
    ("torch-fidelity", "torch_fidelity"),
    ("clip", "clip"),
    ("taming-transformers", "taming"),  # sometimes import is `taming`; if missing we'll still show dist version
    ("rcg", None),                   # your editable -e . package name might differ; we'll try dist lookup
]

def get_dist_ver(dist_name: str):
    try:
        return dist_version(dist_name)
    except PackageNotFoundError:
        return None

def get_import_ver(mod_name: str):
    try:
        m = importlib.import_module(mod_name)
        return getattr(m, "__version__", None)
    except Exception:
        return None

def show_pkg(dist_name, mod_name):
    dv = get_dist_ver(dist_name) if dist_name else None
    mv = get_import_ver(mod_name) if mod_name else None
    v = dv or mv
    status = v if v else "NOT INSTALLED"
    extra = ""
    if dist_name == "pytorch" and "torch" in sys.modules:
        import torch
        extra = f" | cuda={torch.version.cuda} | torch.cuda.is_available()={torch.cuda.is_available()}"
    print(f"{dist_name:18s} -> {status}{extra}")

print("=== Package versions (pip/metadata + import fallback) ===")
for dist_name, mod_name in PKGS:
    if dist_name == "python":
        print(f"{'python':18s} -> {sys.version.split()[0]}")
        continue
    show_pkg(dist_name, mod_name)

print("\n=== GPU / Torch details (if torch installed) ===")
try:
    import torch
    print("torch:", torch.__version__)
    print("torch.version.cuda:", torch.version.cuda)
    print("torch.backends.cudnn.version():", torch.backends.cudnn.version())
    print("cuda available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("device count:", torch.cuda.device_count())
        print("device 0:", torch.cuda.get_device_name(0))
except Exception as e:
    print("torch not usable:", e)

print("\n=== Conda list (filtered) if conda exists ===")
try:
    out = subprocess.check_output(["conda", "list"], text=True)
    # filter lines that mention our key packages
    keys = ["python", "pip", "cudatoolkit", "pytorch", "torchvision", "numpy"]
    for line in out.splitlines():
        if any(k in line.split()[:1] for k in keys):
            print(line)
except Exception as e:
    print("conda not available in this kernel (that's okay):", e)


Python: 3.9.23
Executable: /scratch/gilbreth/abelde/Thesis/StructureAwareGen/SegmentationAwareGen/bin/python

=== Package versions (pip/metadata + import fallback) ===
python             -> 3.9.23
pip                -> 25.3
cudatoolkit        -> NOT INSTALLED
pytorch            -> 2.7.0+cu126 | cuda=12.6 | torch.cuda.is_available()=True
torchvision        -> 0.22.0
numpy              -> 1.20.3
albumentations     -> NOT INSTALLED
opencv-python      -> 4.12.0
pudb               -> NOT INSTALLED
imageio            -> 2.37.0
imageio-ffmpeg     -> 0.4.3
pytorch-lightning  -> 2.5.1.post0
torchmetrics       -> 1.7.2
setuptools         -> 80.9.0
omegaconf          -> 2.3.0
test-tube          -> NOT INSTALLED
streamlit          -> NOT INSTALLED
einops             -> 0.8.1
more-itertools     -> 10.3.0
timm               -> 1.0.15
protobuf           -> 5.29.4
submitit           -> NOT INSTALLED
diffusers          -> NOT INSTALLED
accelerate         -> 1.6.0
huggingface-hub    -> 0.30.2
transforme